## 寻找市场上的强势股

In [1]:
import a1chemy.data_source as data_source
import pymongo
import operator
import pandas as pd
from tqdm.notebook import tqdm



def roc(ticks):
    def rate(t, day1, day2):
        length = t.raw_data.shape[0]
        if -length >= day1 or -length >= day2:
            return -100
        return 100*(t.close().iloc[day1] - t.close().iloc[day2])/t.close().iloc[day2]
    
    return {
        'name': ticks.name,
        'symbol': ticks.symbol,
        '1D': rate(ticks, -1, -2),
        '5D': rate(ticks, -1, -6),
        '1M': rate(ticks, -1, -21),
        'C/S': rate(ticks, -1, -21),
        'S/M': rate(ticks, -21, -61),
        'M/L': rate(ticks, -61, -121),
    }
def percent_color(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val < 0 else 'green'
    return 'color: %s' % color

mongo_client = pymongo.MongoClient("mongodb://localhost:27017/", username='a1chemy', password='1B2C9046-E3CC-447F-9961-E125759BA44F')
mongo_ticks_client = data_source.MongoTicks(mongo_client)
mongo_tags_client = data_source.MongoTags(mongo_client)

def show(tag_id):
    stocks_tree = mongo_tags_client.tree(id=tag_id)
    stocks = []
    for id, stock_tag in tqdm(stocks_tree.root.children.items()):
        exchange=stock_tag.values['exchange']
        symbol =stock_tag.values['symbol']
        ticks = mongo_ticks_client.find_one(exchange=exchange, symbol=symbol)
        stocks.append(ticks)
    roc_list = []
    for stock_tick in stocks:
        roc_list.append(roc(stock_tick))
    sorted_roc_list = sorted(roc_list, key = operator.itemgetter('C/S', 'S/M', 'M/L'))
    df=pd.DataFrame(sorted_roc_list)

    # df.style
    percent_list = list(df)[2:]
    return df.style.format("{:.2f}", subset=percent_list).applymap(percent_color, subset=percent_list)

In [ ]:
show('CASH_COW')

In [ ]:
show('TOP')

In [2]:
show('ZH_ETF')

,name,symbol,1D,5D,1M,C/S,S/M,M/L
0,科创50ETF,SH588000,-0.42,1.36,-100.00,-100.00,-100.00,-100.00
1,中概互联网ETF,SH513050,1.15,-2.41,-4.32,-4.32,4.26,27.48
2,科技50ETF,SH515750,0.21,2.45,-2.73,-2.73,-2.21,23.83
3,医疗ETF,SH512170,1.13,6.48,-1.82,-1.82,-1.83,29.49
4,生物医药ETF,SH512290,0.78,6.72,-1.32,-1.32,-8.01,27.02
5,创新药,SZ159992,1.31,7.33,-0.98,-0.98,-7.44,27.26
6,电子ETF,SZ159997,0.24,2.80,-0.64,-0.64,0.48,19.52
7,新能源车ETF,SH515030,1.63,2.00,-0.51,-0.51,25.77,17.81
8,十年国债ETF,SH511260,0.34,0.34,-0.33,-0.33,-0.01,-1.34
9,国债ETF,SH511010,0.16,0.19,-0.15,-0.15,0.19,-1.84
